<a href="https://colab.research.google.com/github/harmanbirstudy/LLMsPractice/blob/main/gemini2_0_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=vxF8-ay9Bzk

1) Generic run for gemini 2.0 , key from cola secrets

In [ ]:
pip install google-genai

In [ ]:
from google import genai
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
response = client.models.generate_content(model='gemini-2.0-flash-exp', contents='How does AI work?')
print(response.text)

2) Langchain with gemini

https://python.langchain.com/docs/integrations/llms/google_ai/

https://python.langchain.com/docs/concepts/chat_models/

In [15]:
!pip install --quiet langchain

In [ ]:
!pip install --quiet langchain-google-genai

In [32]:
from langchain_google_genai import GoogleGenerativeAI
from google.colab import userdata
api_key=userdata.get('GEMINI_API_KEY')

In [33]:
model = GoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=api_key)

In [ ]:
question1 = "What is BMW revenue for 2022 break it up per line of business and share the 2024 business prespective? "

# for chunk in model.stream(question1):
#   print(chunk, end="")

print(model.invoke(question1))

Using prompts

In [ ]:
from langchain_core.prompts import PromptTemplate
template = "What is {company} revenue for 2022 break it up per line of business and share the {year} business prespective? "

prompt = PromptTemplate.from_template(template)

chain = prompt | model

# for chunck in chain.stream({"company": "BMW", "year": "2024"}):
#   print(chunck, end="")

print(chain.invoke({"company": "BMW", "year": "2024"}))


More params like limit response and print config etc

In [ ]:
for chunk in model.invoke(question1, stream=True, stop=["growth"], temperature=0.2):
  print(chunk, end="" , flush=True)

In [ ]:
response = model.generate([question1])
#response.generations[0][0].text
response.generations[0][0].generation_info

3) Langchain gemini ChatModel

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=api_key)

anwser = chat_model.invoke([HumanMessage(content=question1)])
print(anwser.content)